In [1]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [2]:
import importlib
import module.input_mutation_path as imp
importlib.reload(imp)

<module 'module.input_mutation_path' from '/mnt/ssd1/home3/aiba/gmp/module/input_mutation_path.py'>

In [4]:
strains = ['B.1.1.7','P.1','BA.2','BA.1.1','BA.1','B.1.617.2','B.1.351','B.1.1.529']
strains = ['B.1.1.7']
#strains = ['P.1','BA.2','BA.1.1','B.1.351','BA.1']
usher_dir = '~/usher_output/'
nmax = 1000
nmax_per_strain = 100000000000000000

names,lengths,base_HGVS_paths = imp.input(strains, usher_dir, nmax=nmax, nmax_per_strain=nmax_per_strain)

import pandas as pd
bunpu_df = pd.read_csv("table_heatmap/250621/table_set/table_set.csv")
codon_df = pd.read_csv('meta_data/codon_mutation4.csv')

[INFO] import: /mnt/ssd1/home3/aiba/usher_output/B.1.1.7/0/mutation_paths_B.1.1.7.tsv
[INFO] 指定されたnmax=1000に達しました。
[INFO] B.1.1.7のデータを読み込みました: 1000 サンプル
[INFO] 読み込み完了: 1000 サンプル


In [5]:
import module.get_feature as gfea
importlib.reload(gfea)

<module 'module.get_feature' from '/mnt/ssd1/home3/aiba/gmp/module/get_feature.py'>

In [6]:
data = gfea.Feature_path_incl_ts(base_HGVS_paths, codon_df, bunpu_df)
for i in range(len(data)):
    print(len(data[i]),data[i])

29 {1: [[1, 'C>T', 'b_14408', 'P>L', 'a_323', 'non-syno', 'nsp12', 'c_2', '59']], 2: [[2, 'A>G', 'b_23403', 'D>G', 'a_614', 'non-syno', 'S', 'c_2', '26']], 3: [[3, 'C>T', 'b_241', 'n>n', 'a_0', 'syno', 'non_coding1', 'c_0', '71']], 4: [[4, 'C>T', 'b_3037', 'F>F', 'a_106', 'syno', 'nsp3', 'c_3', '59']], 5: [[5, 'G>A', 'b_28881', 'R>K', 'a_203', 'non-syno', 'N', 'c_2', '25']], 6: [[6, 'G>A', 'b_28882', 'K>K', 'a_203', 'syno', 'N', 'c_3', '135'], [6, 'G>C', 'b_28883', 'G>R', 'a_204', 'non-syno', 'N', 'c_1', '143']], 7: [[7, 'C>T', 'b_884', 'R>C', 'a_27', 'non-syno', 'nsp2', 'c_1', '317']], 8: [[8, 'T>C', 'b_884', 'C>R', 'a_27', 'non-syno', 'nsp2', 'c_1', '24'], [8, 'G>C', 'b_28280', 'D>H', 'a_3', 'non-syno', 'N', 'c_1', '41'], [8, 'A>T', 'b_28281', 'H>L', 'a_3', 'non-syno', 'N', 'c_2', '46'], [8, 'T>A', 'b_28282', 'L>L', 'a_3', 'syno', 'N', 'c_3', '57']], 9: [[9, 'A>T', 'b_23063', 'N>Y', 'a_501', 'non-syno', 'S', 'c_1', '155']], 10: [[10, 'C>T', 'b_15279', 'H>H', 'a_613', 'syno', 'nsp12',

In [7]:
def extract_keys_in_range(data, start_key, end_key):
    # Filter the dictionary to include only keys within the specified range
    filtered_values = []
    for key, value in data.items():
        if start_key <= key <= end_key:
            for v in value:
                filtered_values.append(v)
    return filtered_values

def separate_train_test(data, train_end=30, ylen=1):
    test_start = train_end + 1  # Adjust for zero-based indexing
    train_data_x = []
    train_data_y = []
    test_data_x = {}
    test_data_y = {}

    for d in data:
        if len(d) < test_start:
            train_data_x.append(extract_keys_in_range(d, 1, len(d) - ylen))
            train_data_y.append(extract_keys_in_range(d, len(d) - ylen + 1, len(d)))
        else:
            train_data_x.append(extract_keys_in_range(d, 1, test_start - 1))
            train_data_y.append(extract_keys_in_range(d, test_start, test_start + ylen - 1))

            for i in range(test_start, len(d) + 1 - ylen + 1):
                if i not in test_data_x:
                    test_data_x[i] = []
                    test_data_y[i] = []
                

                test_data_x[i].append(extract_keys_in_range(d, i - test_start + 1, i - 1))
                test_data_y[i].append(extract_keys_in_range(d, i, i + ylen - 1))

    return train_data_x, train_data_y, test_data_x, test_data_y

In [8]:
train_x, train_y, test_x, test_y = separate_train_test(data, train_end=30,ylen=1)

In [ ]:
print("train_x:", len(train_x))
print("train_y:", len(train_y))
for i in range(31, 31+len(test_x)):
    print(f"test_x[{i}]:", len(test_x[i]), test_x[i])
    print(f"test_y[{i}]:", len(test_y[i]), test_y[i])

train_x: 80
train_y: 80
test_x[31]: 15 [[[1, 'C>T', 'b_14408', 'P>L', 'a_323', 'non-syno', 'nsp12', 'c_2', '59'], [2, 'A>G', 'b_23403', 'D>G', 'a_614', 'non-syno', 'S', 'c_2', '26'], [3, 'C>T', 'b_241', 'n>n', 'a_0', 'syno', 'non_coding1', 'c_0', '71'], [4, 'C>T', 'b_3037', 'F>F', 'a_106', 'syno', 'nsp3', 'c_3', '59'], [5, 'G>A', 'b_28881', 'R>K', 'a_203', 'non-syno', 'N', 'c_2', '25'], [6, 'G>A', 'b_28882', 'K>K', 'a_203', 'syno', 'N', 'c_3', '135'], [6, 'G>C', 'b_28883', 'G>R', 'a_204', 'non-syno', 'N', 'c_1', '143'], [7, 'C>T', 'b_884', 'R>C', 'a_27', 'non-syno', 'nsp2', 'c_1', '317'], [8, 'T>C', 'b_884', 'C>R', 'a_27', 'non-syno', 'nsp2', 'c_1', '24'], [8, 'G>C', 'b_28280', 'D>H', 'a_3', 'non-syno', 'N', 'c_1', '41'], [8, 'A>T', 'b_28281', 'H>L', 'a_3', 'non-syno', 'N', 'c_2', '46'], [8, 'T>A', 'b_28282', 'L>L', 'a_3', 'syno', 'N', 'c_3', '57'], [9, 'A>T', 'b_23063', 'N>Y', 'a_501', 'non-syno', 'S', 'c_1', '155'], [10, 'C>T', 'b_15279', 'H>H', 'a_613', 'syno', 'nsp12', 'c_3', '130'

In [9]:
def extract_feature_sequences(paths,feature_idx,feature_name=None):
    features = []
    for path in paths:
        if feature_name is not None:
            feature = [0]*len(feature_name)
        else:
            feature = []
        for mutation in path:
            if len(mutation) > feature_idx: 
                if feature_name is not None:
                    for fn in feature_name:
                        if fn == mutation[feature_idx]:
                            feature[feature_name.index(fn)] = 1
                else:
                    feature.append(mutation[feature_idx])
            else:
                return print(f"Mutation {mutation} does not have enough features.")
        features.append(feature)
    return features

protein_name = ["non_coding1", "nsp1", "nsp2", "nsp3", "nsp4", "nsp5", "nsp6", "nsp7", "nsp8", "nsp9", "nsp10",
                "nsp12", "nsp13", "nsp14", "nsp15", "nsp16",
                "non_coding2", "S", "non_coding3", "ORF3a", "non_coding4", "E", "non_coding5", "M", "non_coding6", 
                "ORF6", "non_coding7", "ORF7a", "ORF7b", "non_coding8", "ORF8", "non_coding9", "N",
                "non_coding10", "ORF10","non_coding11"]

train_y_protein = extract_feature_sequences(train_y, 6, protein_name)
test_y_protein = {}
for i in range(31, 31 + len(test_y)):
    if i not in test_y_protein:
        test_y_protein[i] = []
    test_y_protein[i] = extract_feature_sequences(test_y[i], 6, protein_name)

train_y_protein2 = extract_feature_sequences(train_y, 6)
test_y_protein2 = {}
for i in range(31, 31 + len(test_y)):
    if i not in test_y_protein2:
        test_y_protein2[i] = []
    test_y_protein2[i] = extract_feature_sequences(test_y[i], 6)

In [23]:
print(len(train_x))
print(len(train_x[0]))
print(len(train_x[0][0]))

1000
39
9


In [ ]:
print("train_y:", len(train_y_protein))
print("train_y:", len(train_y_protein2))
for i in range(31, 31+len(test_y_protein)):
    print(f"test_y[{i}]:", len(test_y_protein[i]))
    print(f"test_y[{i}]:", len(test_y_protein2[i]))
    for j in range(len(test_y_protein[i])):
        if test_y_protein[i][j] != test_y_protein2[i][j]:
            print(f"Mismatch at test_y[{i}][{j}]: {test_y_protein[i][j]}")
            print(f"Mismatch at test_y[{i}][{j}]: {test_y_protein2[i][j]}")

train_y: 80
train_y: 80
test_y[31]: 15
test_y[31]: 15
Mismatch at test_y[31][0]: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
Mismatch at test_y[31][0]: ['ORF10']
Mismatch at test_y[31][1]: [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Mismatch at test_y[31][1]: ['nsp5', 'nsp15']
Mismatch at test_y[31][2]: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Mismatch at test_y[31][2]: ['nsp3']
Mismatch at test_y[31][3]: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Mismatch at test_y[31][3]: ['nsp15']
Mismatch at test_y[31][4]: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Mismatch at test_y[31][4]: ['nsp3', 'nsp3']
Mismatch at test_y[31][5]: [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
def restore_features(binary_features, feature_names):

    restored_features = []
    for feature in binary_features:
        restored = [feature_names[i] for i in range(len(feature)) if feature[i] == 1]
        restored_features.append(restored)
    return restored_features

# 使用例
restored_train_y_protein = restore_features(train_y_protein, protein_name)
print(restored_train_y_protein)
print(train_y_protein)

[['N'], ['nsp1'], ['nsp2'], ['ORF10'], ['ORF3a'], ['nsp3', 'nsp4'], ['S'], ['nsp12', 'S'], ['non_coding11'], ['nsp13'], ['nsp13', 'nsp16', 'S', 'N'], ['S'], ['ORF3a', 'N'], ['ORF3a', 'N'], ['ORF3a', 'N'], ['S'], ['S'], ['nsp3'], ['nsp2', 'nsp10'], ['M'], ['nsp3'], ['nsp3'], ['nsp5', 'nsp15'], ['S'], ['nsp3'], ['nsp15'], ['nsp3'], ['S'], ['nsp3'], ['nsp1'], ['ORF3a'], ['ORF3a'], ['ORF3a', 'M'], ['nsp2', 'S'], ['S'], ['nsp16'], ['N'], ['nsp3'], ['non_coding11'], ['nsp14', 'N'], ['nsp3'], ['nsp3'], ['S'], ['nsp3'], ['nsp13'], ['nsp3'], ['non_coding1'], ['nsp12'], ['S'], ['nsp15'], ['nsp16', 'ORF7a', 'N'], ['nsp6'], ['nsp3'], ['non_coding1', 'nsp12', 'S', 'ORF8'], ['ORF7a'], ['S'], ['non_coding11'], ['nsp9'], ['nsp13'], ['non_coding11'], ['nsp14'], ['nsp3'], ['non_coding1'], ['non_coding1'], ['non_coding1'], ['M'], ['nsp3'], ['nsp1', 'N'], ['ORF3a'], ['S'], ['S'], ['nsp5', 'S', 'ORF6', 'ORF7b'], ['nsp2'], ['M'], ['nsp3', 'nsp6', 'nsp13', 'nsp14', 'S', 'ORF3a', 'E', 'ORF6', 'ORF7b'], ['M'],